In [91]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

data source: https://www.zillow.com/research/data/, options: ZHVI Single-Family Homes Time Series, by Zipcodes

In [4]:
zillow_updated_df = pd.read_csv('SingleFamilyResidence.csv', encoding='ISO-8859-1')

In [5]:
zillow_updated_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
0,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1,337200.0,338200.0,339000.0,...,1050700,1049700,1050800,1055800,1061200,1070000,1083000,1086000,1072000,1056500
1,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,2,210400.0,212200.0,212200.0,...,336700,335900,336000,335600,334200,333400,333500,333500,331800,330100
2,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,3,502900.0,504900.0,506300.0,...,1319300,1320800,1325400,1331900,1338800,1345000,1351100,1349700,1337100,1323500
3,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,95400.0,95600.0,95800.0,...,179300,180200,181000,182100,183400,183100,182000,181800,183300,184300
4,93144,79936,El Paso,TX,El Paso,El Paso County,5,77300.0,77300.0,77300.0,...,126400,126900,127600,128200,128600,128900,128700,128500,129000,129800


In [3]:
zillow_updated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15752 entries, 0 to 15751
Columns: 287 entries, RegionID to 2019-07
dtypes: float64(231), int64(52), object(4)
memory usage: 34.5+ MB


In [9]:
#zillow_updated_df.fillna(zillow_updated_df.mean()).head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
0,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1,337200.0,338200.0,339000.0,...,1050700,1049700,1050800,1055800,1061200,1070000,1083000,1086000,1072000,1056500
1,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,2,210400.0,212200.0,212200.0,...,336700,335900,336000,335600,334200,333400,333500,333500,331800,330100
2,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,3,502900.0,504900.0,506300.0,...,1319300,1320800,1325400,1331900,1338800,1345000,1351100,1349700,1337100,1323500
3,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,95400.0,95600.0,95800.0,...,179300,180200,181000,182100,183400,183100,182000,181800,183300,184300
4,93144,79936,El Paso,TX,El Paso,El Paso County,5,77300.0,77300.0,77300.0,...,126400,126900,127600,128200,128600,128900,128700,128500,129000,129800


In [39]:
cols_info = zillow_updated_df.columns[:7]
cols_info

Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank'],
      dtype='object')

In [40]:
cols_2012 = zillow_updated_df.columns[196:]
cols_2012

Index(['2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
       '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12',
       '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06',
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
       '2018-07', '2018-08', '2018-09', '2018-10', 

In [41]:
zillow_updated_df[zillow_updated_df.isnull().any(axis=1)].head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07
18,62012,11201,New York,NY,New York-Newark-Jersey City,Kings County,19,NaN,NaN,NaN,...,4726000,4633300,4586500,4511500,4429700,4405500,4415800,4433400,4478600,4519600
23,61625,10011,New York,NY,New York-Newark-Jersey City,New York County,24,NaN,NaN,NaN,...,10695700,10286900,10185000,10100400,10032400,10006800,9961100,9790400,9558700,9409000
40,61796,10456,New York,NY,New York-Newark-Jersey City,Bronx County,41,NaN,NaN,NaN,...,422700,435900,445300,445800,446200,445900,437600,432000,433000,431000
96,84613,60611,Chicago,IL,Chicago-Naperville-Elgin,Cook County,97,NaN,NaN,NaN,...,1509000,1518500,1535500,1539500,1538600,1534900,1523500,1507900,1498500,1499700
128,99142,97229,Portland,OR,Portland-Vancouver-Hillsboro,Multnomah County,129,NaN,NaN,NaN,...,572200,575500,581400,585800,585000,581900,579800,579200,579100,578900


In [58]:
zillow_updated_2012_df = zillow_updated_df[cols_2012]
zillow_updated_2012_df = zillow_updated_2012_df.dropna()

In [63]:
#zillow_updated_2012_df.head()

In [60]:
#zillow_updated_2012_df.info()  #15752 zipcodes

In [61]:
#filter dataframe for after 2012
zillow_updated_2012_df = pd.merge(zillow_updated_df[cols_info], zillow_updated_2012_df, 
                                  left_index=True, right_index=True)

In [64]:
#zillow_updated_2012_df.info()  #15752 zipcodes

In [70]:
#zillow_updated_2012_df.info() #15451, 301 zipcodes dropped

In [77]:
#fill missing metro area data
zillow_updated_2012_df['Metro'].fillna('N/S', inplace=True)

In [79]:
#zillow_updated_2012_df.info()

In [80]:
#split train_test split, 2012-01 to 2018-06 for train, 2018-07 to 2019-07
cols_train = zillow_updated_2012_df.columns[7:78]
cols_test = zillow_updated_2012_df.columns[78:]
cols_train, cols_test

(Index(['2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
        '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
        '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
        '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
        '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
        '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
        '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
        '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
        '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
        '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12',
        '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06',
        '2017-07', '2017-08', '2017-09', '2017-10', '2017-11'],
       dtype='object'),
 Index(['2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05',
        '2018-06', '2018

In [82]:
#subset dataframes by train and test
train_df = pd.merge(zillow_updated_2012_df[cols_info], zillow_updated_2012_df[cols_train], 
                    left_index=True, right_index=True)
test_df = pd.merge(zillow_updated_2012_df[cols_info], zillow_updated_2012_df[cols_test], 
                    left_index=True, right_index=True)

In [85]:
#melt train_data
train_melt_df = pd.melt(train_df, 
                        id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName', 'RegionID', 'SizeRank'], 
                        var_name= 'time')
train_melt_df['time'] = pd.to_datetime(train_melt_df['time'], infer_datetime_format=True)

In [88]:
#melt train_data
test_melt_df = pd.melt(test_df, 
                        id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName', 'RegionID', 'SizeRank'], 
                        var_name= 'time')
test_melt_df['time'] = pd.to_datetime(test_melt_df['time'], infer_datetime_format=True)

In [92]:
#save to pickles
with open('train.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(train_melt_df, f, pickle.HIGHEST_PROTOCOL)
    
with open('test.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(test_melt_df, f, pickle.HIGHEST_PROTOCOL)

In [96]:
train_melt_df.head(15)

,RegionName,City,State,Metro,CountyName,RegionID,SizeRank,time,value
0,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,84654,1,2012-01-01,739600.0
1,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,91982,2,2012-01-01,248800.0
2,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,84616,3,2012-01-01,931300.0
3,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,91940,4,2012-01-01,116000.0
4,79936,El Paso,TX,El Paso,El Paso County,93144,5,2012-01-01,112400.0
5,77084,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,91733,6,2012-01-01,113300.0
6,10467,New York,NY,New York-Newark-Jersey City,Bronx County,61807,7,2012-01-01,304900.0
7,11226,New York,NY,New York-Newark-Jersey City,Kings County,62037,8,2012-01-01,541500.0
8,60640,Chicago,IL,Chicago-Naperville-Elgin,Cook County,84640,9,2012-01-01,567000.0
9,94109,San Francisco,CA,San Francisco-Oakland-Hayward,San Francisco County,97564,10,2012-01-01,2243000.0
